In [1]:
%matplotlib qt
import dataloader
import pandas as pd
import os
import numpy as np
sid = '08db4255286f'

In [ ]:
acc_data = dataloader.acc_data_for_child(sid)
events = pd.read_csv('data/train_events.csv')
dataloader.annotate_sid(acc_data, events, sid)

In [8]:
acc_data.sample(5)

,series_id,step,timestamp,anglez,enmo,activity
207178,08db4255286f,207178,2018-11-17 13:44:50+00:00,-24.3344,0.0093,Waking
45394,08db4255286f,45394,2018-11-08 05:02:50+00:00,-9.5801,0.0074,Sleeping
390108,08db4255286f,390108,2018-11-28 03:49:00+00:00,-22.3652,0.0443,Waking
329363,08db4255286f,329363,2018-11-24 15:26:55+00:00,-30.3950,0.0003,Sleeping
176233,08db4255286f,176233,2018-11-15 18:46:05+00:00,-11.8406,0.0099,Waking


In [26]:
def filter_data(acc_data:pd.DataFrame, tss: list[pd.Timestamp], window: pd.Timedelta):
    net = sum([(acc_data['timestamp'] >= ts-window) & (acc_data['timestamp'] < ts+window) for ts in tss])
    net = net==True
    return acc_data.loc[net]

In [28]:
filt_acc = filter_data(
    acc_data=acc_data,
    tss=pd.to_datetime(events.loc[events['series_id']==sid, 'timestamp'], utc=True),
    window=pd.Timedelta(1, 'hour')
)

In [65]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm.notebook import tqdm
from visual import Visualizaer1
fig, ax = plt.subplots()

artists = Visualizaer1.init_frame(ax, filt_acc)
print(ax.get_yticklabels())
ani = anim.FuncAnimation(
    func=Visualizaer1.draw_frame,
    fig=fig,
    frames=tqdm(filt_acc['timestamp'][::30]),
    fargs=(filt_acc, pd.Timedelta(20, 'min'), acc_data['anglez'].std(), 2*acc_data['enmo'].std(), *artists),
    blit=True,
    interval=100

)
plt.show()



[Text(0, -3.0, '−3.0'), Text(0, -2.5, '−2.5'), Text(0, -2.0, '−2.0'), Text(0, -1.5, '−1.5'), Text(0, -1.0, '−1.0'), Text(0, -0.5, '−0.5'), Text(0, 0.0, '0.0'), Text(0, 0.5, '0.5'), Text(0, 1.0, '1.0')]


  0%|          | 0/2400 [00:00<?, ?it/s]

In [72]:
import importlib
import visual
importlib.reload(visual)

<module 'visual' from 'd:\\Projects\\ChildMindSleep\\visual.py'>

In [74]:
from visual import Visualizer2

fig, ax = plt.subplots()
time_window = pd.Timedelta(10, 'min')
artists = Visualizer2.init_frame(ax, time_window)
frames = tqdm(filt_acc['timestamp'][::30])
ani = anim.FuncAnimation(
    fig=fig,
    func=Visualizer2.draw_frame,
    frames=frames,
    fargs=(filt_acc, 0.3, time_window, artists),
    blit=True,
    interval=100
)
# ani.save('vids/filtered_activity.mp4')


  0%|          | 0/2400 [00:00<?, ?it/s]